In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import nltk

In [2]:
col = ['target','ids','date','flag','user','tweets']

In [3]:
twitter_data = pd.read_csv(f'/home/sonis/datasets/training.1600000.processed.noemoticon.csv',encoding='ISO-8859-1')
twitter_data.columns  = col
twitter_data.head()

,target,ids,date,flag,user,tweets
0,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
1,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
2,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
3,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
4,0,1467811372,Mon Apr 06 22:20:00 PDT 2009,NO_QUERY,joy_wolf,@Kwesidei not the whole crew


In [4]:
data= twitter_data[['target','tweets']]
data.head()

,target,tweets
0,0,is upset that he can't update his Facebook by ...
1,0,@Kenichan I dived many times for the ball. Man...
2,0,my whole body feels itchy and like its on fire
3,0,"@nationwideclass no, it's not behaving at all...."
4,0,@Kwesidei not the whole crew


In [5]:
data.target.replace(4,1,inplace=True)

/tmp/ipykernel_15055/3469991616.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.target.replace(4,1,inplace=True)


In [6]:
data.target.value_counts()

1    800000
0    799999
Name: target, dtype: int64

In [14]:
pos = data[data['target']==1]
neg = data[data['target']==0]
pos.head(),neg.head()


(        target                                             tweets
 799999       1       I LOVE @Health4UandPets u guys r the best!! 
 800000       1  im meeting up with one of my besties tonight! ...
 800001       1  @DaRealSunisaKim Thanks for the Twitter add, S...
 800002       1  Being sick can be really cheap when it hurts t...
 800003       1    @LovesBrooklyn2 he has that effect on everyone ,
    target                                             tweets
 0       0  is upset that he can't update his Facebook by ...
 1       0  @Kenichan I dived many times for the ball. Man...
 2       0    my whole body feels itchy and like its on fire 
 3       0  @nationwideclass no, it's not behaving at all....
 4       0                      @Kwesidei not the whole crew )

In [16]:
pos.count(),neg.count()


(target    800000
 tweets    800000
 dtype: int64,
 target    799999
 tweets    799999
 dtype: int64)

In [17]:
train_pos = pos[400000:] 
test_pos = pos[:400000]
train_neg = neg[400000:]
test_neg = neg[:400000]

In [18]:
train_pos

,target,tweets
1199999,1,danielle is the best thing since cheese in a c...
1200000,1,@roxyreeb You need a new hobby. Wanna train fo...
1200001,1,"loooves my friends, and as of today loves the ..."
1200002,1,the hills last one D:
1200003,1,teamviewer is entertaining
...,...,...
1599994,1,Just woke up. Having no school is the best fee...
1599995,1,TheWDB.com - Very cool to hear old Walt interv...
1599996,1,Are you ready for your MoJo Makeover? Ask me f...
1599997,1,Happy 38th Birthday to my boo of alll time!!! ...


In [34]:
train_x = train_pos['tweets'].tolist()+train_neg['tweets'].tolist()
# train_y = train_pos['target']+train_neg['target']
test_x = test_pos['tweets'].tolist()+test_neg['tweets'].tolist()
# test_y = test_pos['target']+test_neg['target']

train_y = np.append(np.ones(len(train_pos)), np.zeros(len(train_neg)))
test_y = np.append(np.ones(len(test_pos)), np.zeros(len(test_neg)))

In [35]:
train_x

['danielle is the best thing since cheese in a can! ',
 '@roxyreeb You need a new hobby. Wanna train for a triathlon with me? ',
 'loooves my friends, and as of today loves the chillness of sundays! ',
 'the hills  last one D:',
 'teamviewer is entertaining ',
 'i intensely miss my bestfriends! much love!!  [two more days]',
 "Watching the Hills season finale!!! Can't wait for the world premier of the New Moon clip!!!! ",
 'The Hills and MTV music awards  what a way to end my weekend!',
 '@KempEquine thanks  It was a lot of fun!',
 "@JerZDre nothin much doin' the same--&gt;its a Sunday I feel like I'm breaking a rule by doin' anything today besides chill ",
 '@KempEquine thanks  It was a lot of fun!',
 'is lovin the sun ',
 "just one hour left to the awards  ... watchin' the red carpet right now! @ashleytisdale u look amazing ",
 '80.6 Fahrenheit tomorrow  Going to the beach  this has been the most amazing week its seldom every warm in Scotland',
 '@gypsymoondesign Way to go!  I am mor

In [36]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /home/sonis/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [9]:
import re                                  # library for regular expression operations
import string                              # for string operations

from nltk.corpus import stopwords          # module for stop words that come with NLTK
from nltk.stem import PorterStemmer        # module for stemming
from nltk.tokenize import TweetTokenizer   # module for tokenizing strings

In [10]:
stopwords_english = stopwords.words('english') 

print('Stop words\n')
print(stopwords_english)

print('\nPunctuation\n')
print(string.punctuation)

Stop words

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so

In [11]:
def process_tweet(tweet):
    stemmer = PorterStemmer()
    stopwords_english = stopwords.words('english')
    tweet = re.sub(r'\$\w*', '', tweet)
    tweet = re.sub(r'^RT[\s]+', '', tweet)
    tweet = re.sub(r'https?:\/\/.*[\r\n]*', '', tweet)
    tweet = re.sub(r'#', '', tweet)
    tokenizer = TweetTokenizer(preserve_case=False,        strip_handles=True,reduce_len=True)
    tweet_tokens = tokenizer.tokenize(tweet)

    tweets_clean = []
    for word in tweet_tokens:
        if (word not in stopwords_english and  
                word not in string.punctuation): 
            stem_word = stemmer.stem(word)  # stemming word
            tweets_clean.append(stem_word)

    return tweets_clean

In [23]:
custom_tweet = "RT @Twitter @chapagain Hello There! Have a great day. :) #good #morning http://chapagain.com.np"

# print cleaned tweet
print(process_tweet(custom_tweet))

['hello', 'great', 'day', ':)', 'good', 'morn']


In [24]:
# UNQ_C1 (UNIQUE CELL IDENTIFIER, DO NOT EDIT)
def count_tweets(result, tweets, ys):
    '''
    Input:
        result: a dictionary that will be used to map each pair to its frequency
        tweets: a list of tweets
        ys: a list corresponding to the sentiment of each tweet (either 0 or 1)
    Output:
        result: a dictionary mapping each pair to its frequency
    '''

    ### START CODE HERE (REPLACE INSTANCES OF 'None' with your code) ###
    for y, tweet in zip(ys, tweets):
        for word in process_tweet(tweet):
            # define the key, which is the word and label tuple
            pair = (word,y)

            # if the key exists in the dictionary, increment the count
            if pair in result:
                result[pair] += 1

            # else, if the key is new, add it to the dictionary and set the count to 1
            else:
                result[pair] = 1
    ### END CODE HERE ###

    return result

In [25]:
# Testing your function


result = {}
tweets = ['i am happy', 'i am tricked', 'i am sad', 'i am tired', 'i am tired']
ys = [1, 0, 0, 0, 0]
count_tweets(result, tweets, ys)

{('happi', 1): 1, ('trick', 0): 1, ('sad', 0): 1, ('tire', 0): 2}

In [37]:
# Build the freqs dictionary for later uses

freqs = count_tweets({}, train_x, train_y)

In [52]:
len(freqs),len(set(freqs.keys())),len(train_x)

(217263, 217263, 799999)

In [56]:
def lookup(freqs, word, label):
    '''
    Input:
        freqs: a dictionary with the frequency of each pair (or tuple)
        word: the word to look up
        label: the label corresponding to the word
    Output:
        n: the number of times the word with its corresponding label appears.
    '''
    n = 0  # freqs.get((word, label), 0)

    pair = (word, label)
    if (pair in freqs):
        n = freqs[pair]

    return n

In [74]:
vocab = set(pair[0] for pair in freqs.keys())
len(vocab)

172495

In [58]:
freq_pos

0

In [71]:
# UNQ_C2 (UNIQUE CELL IDENTIFIER, DO NOT EDIT)
def train_naive_bayes(freqs, train_x, train_y):
    '''
    Input:
        freqs: dictionary from (word, label) to how often the word appears
        train_x: a list of tweets
        train_y: a list of labels correponding to the tweets (0,1)
    Output:
        logprior: the log prior. (equation 3 above)
        loglikelihood: the log likelihood of you Naive bayes equation. (equation 6 above)
    '''
    loglikelihood = {}
    logprior = 0

    ### START CODE HERE (REPLACE INSTANCES OF 'None' with your code) ###

    # calculate V, the number of unique words in the vocabulary
    vocab = set([pair[0] for pair in freqs.keys()])
    V = len(vocab)

    # calculate N_pos, N_neg, V_pos, V_neg
    N_pos = N_neg = V_pos = V_neg = 0
    for pair in freqs.keys():
        # if the label is positive (greater than zero)
        if pair[1] > 0:
            # increment the count of unique positive words by 1
            V_pos += 1

            # Increment the number of positive words by the count for this (word, label) pair
            N_pos += freqs[pair]

        # else, the label is negative
        else:
            # increment the count of unique negative words by 1
            V_neg += 1

            # increment the number of negative words by the count for this (word,label) pair
            N_neg += freqs[pair]

    # Calculate D, the number of documents
    D = len(train_y)

    # Calculate D_pos, the number of positive documents
    D_pos = (len(list(filter(lambda x: x > 0, train_y))))

    # Calculate D_neg, the number of negative documents
    D_neg = (len(list(filter(lambda x: x <= 0, train_y))))

    # Calculate logprior
    logprior = np.log(D_pos) - np.log(D_neg)

    # For each word in the vocabulary...
    for word in vocab:
        # get the positive and negative frequency of the word
        freq_pos = lookup(freqs,word,1)
        freq_neg = lookup(freqs,word,0)

        # calculate the probability that each word is positive, and negative
        p_w_pos = (freq_pos + 1) / (N_pos + V)
        p_w_neg = (freq_neg + 1) / (N_neg + V)

        # calculate the log likelihood of the word
        loglikelihood[word] = np.log(p_w_pos/p_w_neg)

    ### END CODE HERE ###

    return logprior, loglikelihood


In [72]:
logprior, loglikelihood = train_naive_bayes(freqs, train_x, train_y)
print(logprior)
print(len(loglikelihood))

2.5000031254052146e-06
172495


In [75]:
# UNQ_C4 (UNIQUE CELL IDENTIFIER, DO NOT EDIT)
def naive_bayes_predict(tweet, logprior, loglikelihood):
    '''
    Input:
        tweet: a string
        logprior: a number
        loglikelihood: a dictionary of words mapping to numbers
    Output:
        p: the sum of all the logliklihoods of each word in the tweet (if found in the dictionary) + logprior (a number)

    '''
    ### START CODE HERE (REPLACE INSTANCES OF 'None' with your code) ###
    # process the tweet to get a list of words
    word_l = process_tweet(tweet)

    # initialize probability to zero
    p = 0

    # add the logprior
    p += logprior

    for word in word_l:

        # check if the word exists in the loglikelihood dictionary
        if word in loglikelihood:
            # add the log likelihood of that word to the probability
            p += loglikelihood[word]

    ### END CODE HERE ###

    return p


In [79]:
my_tweet = 'She cried.'
p = naive_bayes_predict(my_tweet, logprior, loglikelihood)
print('The expected output is', p)

The expected output is -1.799405334124454


In [80]:
# UNQ_C6 (UNIQUE CELL IDENTIFIER, DO NOT EDIT)
def test_naive_bayes(test_x, test_y, logprior, loglikelihood):
    """
    Input:
        test_x: A list of tweets
        test_y: the corresponding labels for the list of tweets
        logprior: the logprior
        loglikelihood: a dictionary with the loglikelihoods for each word
    Output:
        accuracy: (# of tweets classified correctly)/(total # of tweets)
    """
    accuracy = 0  # return this properly

    ### START CODE HERE (REPLACE INSTANCES OF 'None' with your code) ###
    y_hats = []
    for tweet in test_x:
        # if the prediction is > 0
        if naive_bayes_predict(tweet, logprior, loglikelihood) > 0:
            # the predicted class is 1
            y_hat_i = 1
        else:
            # otherwise the predicted class is 0
            y_hat_i = 0

        # append the predicted class to the list y_hats
        y_hats.append(y_hat_i)

    # error is the average of the absolute values of the differences between y_hats and test_y
    error = np.mean(np.absolute(y_hats-test_y))

    # Accuracy is 1 minus the error
    accuracy = 1-error

    ### END CODE HERE ###

    return accuracy


In [81]:
print("Naive Bayes accuracy = %0.4f" %
      (test_naive_bayes(test_x, test_y, logprior, loglikelihood)))

Naive Bayes accuracy = 0.7550


In [82]:
def get_ratio(freqs, word):
    '''
    Input:
        freqs: dictionary containing the words

    Output: a dictionary with keys 'positive', 'negative', and 'ratio'.
        Example: {'positive': 10, 'negative': 20, 'ratio': 0.5}
    '''
    pos_neg_ratio = {'positive': 0, 'negative': 0, 'ratio': 0.0}
    ### START CODE HERE (REPLACE INSTANCES OF 'None' with your code) ###
    # use lookup() to find positive counts for the word (denoted by the integer 1)
    pos_neg_ratio['positive'] = lookup(freqs,word,1)

    # use lookup() to find negative counts for the word (denoted by integer 0)
    pos_neg_ratio['negative'] = lookup(freqs,word,0)

    # calculate the ratio of positive to negative counts for the word
    pos_neg_ratio['ratio'] = (pos_neg_ratio['positive'] + 1)/(pos_neg_ratio['negative'] + 1)
    ### END CODE HERE ###
    return pos_neg_ratio


In [83]:
# UNQ_C9 (UNIQUE CELL IDENTIFIER, DO NOT EDIT)
def get_words_by_threshold(freqs, label, threshold):
    '''
    Input:
        freqs: dictionary of words
        pos_neg_ratio: dictionary of positive counts, negative counts, and ratio of positive / negative counts.
        label: 1 for positive, 0 for negative
        threshold: ratio that will be used as the cutoff for including a word in the returned dictionary
    Output:
        word_set: dictionary containing the word and information on its positive count, negative count, and ratio of positive to negative counts.
        example of a key value pair:
        {'happi':
            {'positive': 10, 'negative': 20, 'ratio': 0.5}
        }
    '''
    word_list = {}

    ### START CODE HERE (REPLACE INSTANCES OF 'None' with your code) ###
    for key in freqs.keys():
        word, _ = key

        # get the positive/negative ratio for a word
        pos_neg_ratio = get_ratio(freqs, word)

        # if the label is 1 and the ratio is greater than or equal to the threshold...
        if label == 1 and pos_neg_ratio['ratio'] >= threshold :

            # Add the pos_neg_ratio to the dictionary
            word_list[word] = pos_neg_ratio

        # If the label is 0 and the pos_neg_ratio is less than or equal to the threshold...
        elif label == 0 and pos_neg_ratio['ratio'] <= threshold:
            # Add the pos_neg_ratio to the dictionary
            word_list[word] = pos_neg_ratio

        # otherwise, do not include this word in the list (do nothing)

    ### END CODE HERE ###
    return word_list


In [91]:
# Test your function: find negative words at or below a threshold
get_words_by_threshold(freqs, label=0, threshold=0.05)

{'farrah': {'positive': 7, 'negative': 506, 'ratio': 0.015779092702169626},
 'unlov': {'positive': 1, 'negative': 52, 'ratio': 0.03773584905660377},
 'devast': {'positive': 4, 'negative': 118, 'ratio': 0.04201680672268908},
 'sadfac': {'positive': 2, 'negative': 65, 'ratio': 0.045454545454545456},
 'os3': {'positive': 3, 'negative': 84, 'ratio': 0.047058823529411764},
 'twug': {'positive': 3, 'negative': 126, 'ratio': 0.031496062992125984},
 'mcmahon': {'positive': 2, 'negative': 198, 'ratio': 0.01507537688442211},
 'sadd': {'positive': 1, 'negative': 62, 'ratio': 0.031746031746031744},
 'heartbroken': {'positive': 2, 'negative': 119, 'ratio': 0.025},
 'saaad': {'positive': 1, 'negative': 42, 'ratio': 0.046511627906976744},
 'boohoo': {'positive': 2, 'negative': 90, 'ratio': 0.03296703296703297},
 'saddd': {'positive': 0, 'negative': 82, 'ratio': 0.012048192771084338},
 'saad': {'positive': 0, 'negative': 21, 'ratio': 0.045454545454545456},
 'tummyach': {'positive': 0, 'negative': 26, 

In [94]:
# Test your function; find positive words at or above a threshold
get_words_by_threshold(freqs, label=1, threshold=8)

{'mtv': {'positive': 996, 'negative': 95, 'ratio': 10.385416666666666},
 'followfriday': {'positive': 787,
  'negative': 61,
  'ratio': 12.709677419354838},
 'welcom': {'positive': 3262, 'negative': 300, 'ratio': 10.840531561461795},
 'gentlemen': {'positive': 35, 'negative': 3, 'ratio': 9.0},
 'fuzzbal': {'positive': 53, 'negative': 1, 'ratio': 27.0},
 'x17live': {'positive': 19, 'negative': 0, 'ratio': 20.0},
 '¹': {'positive': 788, 'negative': 73, 'ratio': 10.662162162162161},
 'congrat': {'positive': 1765, 'negative': 213, 'ratio': 8.25233644859813},
 'mooon': {'positive': 18, 'negative': 1, 'ratio': 9.5},
 'maneat': {'positive': 8, 'negative': 0, 'ratio': 9.0},
 '\x84': {'positive': 787, 'negative': 92, 'ratio': 8.473118279569892},
 'howdi': {'positive': 77, 'negative': 7, 'ratio': 9.75},
 'e3': {'positive': 407, 'negative': 26, 'ratio': 15.11111111111111},
 'frasier': {'positive': 7, 'negative': 0, 'ratio': 8.0},
 'breakthrough': {'positive': 26, 'negative': 1, 'ratio': 13.5},
 '

In [95]:
# Some error analysis done for you
print('Truth Predicted Tweet')
for x, y in zip(test_x, test_y):
    y_hat = naive_bayes_predict(x, logprior, loglikelihood)
    if y != (np.sign(y_hat) > 0):
        print('%d\t%0.2f\t%s' % (y, np.sign(y_hat) > 0, ' '.join(
            process_tweet(x)).encode('ascii', 'ignore')))

Truth Predicted Tweet
1	0.00	b'sick realli cheap hurt much eat real food plu friend make soup'
1	0.00	b'than respons ihad alreadi find answer'
1	0.00	b'respond stupid cat help type forgiv error'
1	0.00	b'least one feel lost may caus mani later usual night alreadi addict'
1	0.00	b'danc around room pj jam ipod get dizzi well twitter ask'
1	0.00	b'walk put deposit tomorrow'
1	0.00	b'reach amritsar hour find bu wagah border 2pm'
1	0.00	b'final go bed tire gonna watch hill didnt =/'
1	0.00	b'ef tire throat hurt ... ooohh ... got crazi crave pina colada banana slushi'
1	0.00	b"awww pooki feel better i'll pray ... bag nurs"
1	0.00	b'think met first snob twitter tonight bad life goe'
1	0.00	b'long convers mom phone'
1	0.00	b'practic line manner voic upcom featur shoot probabl drive brock crazi'
1	0.00	b'bore'
1	0.00	b'thank respons ihad alreadi find answer'
1	0.00	b"well yeah notic cuz i'm bb"
1	0.00	b"can't get earli enough night workout much better"
1	0.00	b"sorrri i'm like 2982378 2 differ 

1	0.00	b'almost late work ten minut get readi lack sleep consequ factori friday beachbar sunday'
1	0.00	b'lose time'
1	0.00	b'hit myspac im goin sleep tho peac www.myspace.com/dxckinyamouth'
1	0.00	b"... watch preview they'v play way much ... look realli bad stop watch haha ..."
1	0.00	b'four day till get see girlfriend'
1	0.00	b'still 1st coffe day happi tuesday see jeff dunham achm dead terrorist tonight'
1	0.00	b'home time chelsea late'
1	0.00	b'earli work today tomorrow cours thursday friday bad'
1	0.00	b'oh ish kid cali'
1	0.00	b'u move back jersey gonna better'
1	0.00	b'great episod didnt know would two'
1	0.00	b'got tweet sorri distract two heavenli bodi'
1	0.00	b'back school school two week'
1	0.00	b'goodnight <3 goin bed gotta get earli get fuck town tomorrow canada come'
1	0.00	b'lie bed babe'
1	0.00	b'still playin card girl'
1	0.00	b'could one miniatur'
1	0.00	b'mac could anyth'
1	0.00	b'dont neither'
1	0.00	b'instal twitterberri blackberri bold'
1	0.00	b'shazzam vivvid 78 g

1	0.00	b'know real though lot fake'
1	0.00	b'almost want one'
1	0.00	b'weight lost healthi eat slim world'
1	0.00	b'hope get good sleep tonight ... poor babi still feel bad nice warm lavend bath made feel good though tough'
1	0.00	b"yeah long there' still ticket free"
1	0.00	b'love lot work need keep occupi next 2 day'
1	0.00	b'teeni bit date leprechaun'
1	0.00	b"ridicul hot apart .. can't wait weekend"
1	0.00	b'hugg back ... think hug man twitter'
1	0.00	b'busi busi night fold laundri pack write paper ... back portland 48 hour'
1	0.00	b'well ... hectic busi work famili'
1	0.00	b'one heart'
1	0.00	b'beauti morn shame im home sleep'
1	0.00	b'tri get done research go away weekend ...'
1	0.00	b'e pain subsid enough hurt'
1	0.00	b'go bed littl earli today ... whole half hour earli ... lol'
1	0.00	b'motorbik much fun electr bike rode around citi one yesterday new summer activ think'
1	0.00	b"ummm ... think use far dunno made 15 min ago thought it'd fun"
1	0.00	b'know mean look bounc back se

1	0.00	b'agre entir wait much longer'
1	0.00	b'eww math one day till holiday'
1	0.00	b'hard time life night kept aliv'
1	0.00	b'less hour sleep nap hour sleep'
1	0.00	b'work hard work smart'
1	0.00	b'know happen'
1	0.00	b'anoth long day begin jame glad sunni stay away nasti rain'
1	0.00	b'come stay mine friday'
1	0.00	b'encantado .. look stuff like past day ..'
1	0.00	b'wish ex red arrow privileg fli'
1	0.00	b'last call soon slow night'
1	0.00	b'ugh first know fault make stay late monday'
1	0.00	b'got'
1	0.00	b'still think twitter hipster circl jerk'
1	0.00	b'ugh hate got snow middl april ... dont miss canada end sarcasm'
1	0.00	b'im work return hello get home'
1	0.00	b'tell spring break boy school first light actual stay late'
1	0.00	b"hunni vampir ... bloodsuck salespeopl steal daddy' money ... day 1 lycan"
1	0.00	b'still'
1	0.00	b'excit suppos 75 degre tomorrow yeah'
1	0.00	b'watch spongebob till fall asleep'
1	0.00	b'download ici let know think watch'
1	0.00	b'omg sooo cannot wait 

1	0.00	b'tire final get sleep nighti'
1	0.00	b"essay finish i'm go hooom"
1	0.00	b'know still shit sorri mean shift'
1	0.00	b"actual i'v sleep 6 hour good 6 hour sleep feel fine i'm goood"
1	0.00	b'around mid may 3 week'
1	0.00	b"can't wait tomorrow"
1	0.00	b'ebtm.com back'
1	0.00	b'long day hope catch zz result grab blankey chair gotta b bit 4 bloodlet'
1	0.00	b"good morn internet ipod touch might come today even i'm use sunday"
1	0.00	b'want go basebal game doesnt care ... someon take'
1	0.00	b'bad true part'
1	0.00	b'anoth day anoth ... coach load brummi'
1	0.00	b'awak ... phone txt'
1	0.00	b"10 blister last night' gig 4 turn turn ulcer sort still gig tonight"
1	0.00	b'want find time scrapbook'
1	0.00	b'night everywher daylight ..'
1	0.00	b"chriiist i'm tire ... old shizzit nowaday think go get cup joe wake"
1	0.00	b'stick around ... soon unveil hacker evolut iphon ipod'
1	0.00	b'happen load repli id ramdomthought randomthought spell wrong deliber'
1	0.00	b'go bed night'
1	0.00	b"cl

...

1	0.00	b'back offic'
1	0.00	b"i'm leav .. byeee"
1	0.00	b'offic someon want buy hous utrecht'
1	0.00	b'doubl degre inshaallah'
1	0.00	b'yell lord know mani driver respect funer process oy prankin grave awesom'
1	0.00	b'consid tri milo dinosaur next time milo ice heap spoon milo powder possibl horlick'
1	0.00	b'worri ... u like oh btw ac thing problem either'
1	0.00	b'one day chillout munich tomorrow'
1	0.00	b'would'
1	0.00	b'lol relaiz cant spell .. got new phone troubl adapt'
1	0.00	b'yummi gunna make noodl goreng spell right anyon wanna help math'
1	0.00	b'alway miss u .. love'
1	0.00	b'api trial'
1	0.00	b'got home visit templ realli realli hot outsid get readi go lunch shop'
1	0.00	b"i'm close almost tast adventur"
1	0.00	b'pray pour rain yetserday god spiritu rain fall youth nation ... amaz experi'
1	0.00	b'bahahahahaha suck ili'
1	0.00	b'yesterday 1st perform choir genr tearjerk today work take easi bad cold :/'
1	0.00	b'bamaloo doesnt like'
1	0.00	b'rememb regret .. haoppen reaso

1	0.00	b'lost mine work wknd bed give nite nite'
1	0.00	b'make cash ms hatti'
1	0.00	b'dude want astronaut even went space camp'
1	0.00	b'ive sick past day that think may watch origin seri pass time'
1	0.00	b'dod feel satisfact goe'
1	0.00	b'final yawn ... must bed time'
1	0.00	b'decid phone call make 2mr sorta piss right lol blip power'
1	0.00	b'widget market may well stop act like custom understand widget'
1	0.00	b"digsbi work school ye i'm back msn"
1	0.00	b'yea season 2'
1	0.00	b'fun play 4pm humid condit tri'
1	0.00	b'went old comment old pli'
1	0.00	b'bad signal remov script full power finish bad signal ping'
1	0.00	b'half way first day internship bad far'
1	0.00	b'nigga u know ur window aint got screen plzzz dont kill urself ova song lol ... take night'
1	0.00	b'u want dev go home'
1	0.00	b"sorri guy tweet much i'm get david archi repli"
1	0.00	b"u wanna talk bout i'm"
1	0.00	b'run skirrid fawr tire happi'
1	0.00	b'catch lost brilliant lost'
1	0.00	b'still week girl take easi br

1	0.00	b"lol right gotta tri get home i'm use work x"
1	0.00	b"head wmc day hope i'll enough time swing chinatown way home i'm mood duck"
1	0.00	b'tri figur'
1	0.00	b'work reachabl mobil ...'
1	0.00	b'probabl get chilli'
1	0.00	b'well day owe sleep late'
1	0.00	b'everyth noth hmmm'
1	0.00	b'storm system look bring us rain today go miss us'
1	0.00	b'need sleep'
1	0.00	b'funni wish grandma'
1	0.00	b'haha nto 2 lonq aqo qot dh ashow'
1	0.00	b'ehh need break gonna 3pg essay 2m work 5 go bed gym morn nighttt'
1	0.00	b'break rule take tension ...'
1	0.00	b"nope there'"
1	0.00	b'well littl bit sick fine'
1	0.00	b'stop'
1	0.00	b"know like caus i'v never ask bout him.oh well guess right feel abit better"
1	0.00	b'work realli hard sometim need break haha yeah ... let know need help anythin'
1	0.00	b'30 day till p nk babysit 15 minut'
1	0.00	b'well revis garden guess hoover warcraft play week'
1	0.00	b'ah hate happen race eras ms week go'
1	0.00	b'oh total work'
1	0.00	b'hey oyi gea sleep ...'
1	

1	0.00	b'awww shook im gon overprotect incas start cybercheatin lol'
1	0.00	b"well there' noth keep swim get eventu"
1	0.00	b'ummm ... look head'
1	0.00	b'sneez felt like train-wreck last two week piggystuff good'
1	0.00	b'adob effect ms sql server ... nice would nicer sleep'
1	0.00	b'get 3 way call'
1	0.00	b"well ... gotta grab least hour half sleep girl' get school ... gawd life multi-taskin night mornin"
1	0.00	b'godaw hangov today worth tho'
1	0.00	b'gone ear home .. plenti paint well'
1	0.00	b"need ship forecast we'r inland"
1	0.00	b'school today gd dont get see mate n millsi x'
1	0.00	b"i'm elev error rate ... streamkey found connect problem server migrat easi huh"
1	0.00	b'remind get take pic bloom clemati wind storm tomorrow'
1	0.00	b'way inconveni ...'
1	0.00	b'way home school lol car tire'
1	0.00	b"sorri disappoint big nascar fan i'm still all-around decent redneck"


KeyboardInterrupt: 

In [105]:
# Test with your own tweet - feel free to modify `my_tweet`
my_tweet = 'I am sad happy'

p = naive_bayes_predict(my_tweet, logprior, loglikelihood)
print(p)

-1.85131665357578


In [18]:
# # from utils import process_tweet # Import the process_tweet function

# # choose the same tweet
# # tweet = all_positive_tweets[2277]

# print()
# print('\033[92m')
# print(x_train[9])
# print('\033[94m')

# # call the imported function
# tweets_stem = process_tweet(x_train[9]); # Preprocess a given tweet

# print('preprocessed tweet:')
# print(tweets_stem) # Print the result



spring break in plain city... it's snowing 

preprocessed tweet:
['spring', 'break', 'plain', 'citi', '...', 'snow']


In [19]:
# # create frequency dictionary
# freqs = build_freqs(x_train, y_train)

# # check data type
# print(f'type(freqs) = {type(freqs)}')

# # check length of the dictionary
# print(f'len(freqs) = {len(freqs)}')

type(freqs) = <class 'dict'>
len(freqs) = 271884


In [20]:
# freqs

{('like', 1): 27621,
 ('audit', 0): 197,
 ('come', 0): 13851,
 ('exactli', 0): 614,
 ('week', 0): 11046,
 ('allow', 0): 933,
 ('go', 0): 54987,
 ('venu', 0): 122,
 ('heeellppp', 0): 1,
 ('l', 1): 1056,
 ('train', 1): 2264,
 ('canarsi', 1): 3,
 ('rockaway', 1): 7,
 ('prkway', 1): 1,
 ('bound', 1): 161,
 ('2', 1): 11724,
 ('appet', 1): 16,
 ('menu', 1): 98,
 ('salad', 1): 366,
 ('filet', 1): 13,
 ('mignon', 1): 7,
 ('bake', 1): 523,
 ('potato', 1): 262,
 ('creme', 1): 40,
 ('brule', 1): 10,
 ('4', 1): 5374,
 ('cocktail', 1): 135,
 ('tweet', 0): 5973,
 ('4skype', 0): 1,
 ('could', 0): 9324,
 ('nice', 0): 4553,
 ('impement', 0): 1,
 ('work', 0): 40964,
 ('congrat', 1): 2279,
 ('bella', 1): 136,
 ('15', 1): 738,
 ('yr', 1): 660,
 ('could', 1): 5562,
 ('beat', 1): 1149,
 ('boy', 1): 3106,
 ('tour', 1): 1260,
 ('de', 1): 629,
 ('franc', 1): 255,
 ('enrout', 1): 8,
 ('newburi', 1): 9,
 ('spa', 1): 137,
 ('afternoon', 1): 1705,
 ('pamper', 1): 65,
 ('spoil', 1): 207,
 ('alon', 0): 2772,
 ('girl

In [22]:
# def extract_features(tweet, freqs):
#     '''
#     Input: 
#         tweet: a list of words for one tweet
#         freqs: a dictionary corresponding to the frequencies of each tuple (word, label)
#     Output: 
#         x: a feature vector of dimension (1,3)
#     '''
#     # process_tweet tokenizes, stems, and removes stopwords
#     word_l = process_tweet(tweet)
    
#     # 3 elements in the form of a 1 x 3 vector
#     x = np.zeros((1, 3)) 
    
#     #bias term is set to 1
#     x[0,0] = 1 
    
#     ### START CODE HERE (REPLACE INSTANCES OF 'None' with your code) ###
    
#     # loop through each word in the list of words
#     for word in word_l:
        
#         # increment the word count for the positive label 1
#         x[0,1] += freqs.get((word, 1.0),0)
        
#         # increment the word count for the negative label 0
#         x[0,2] += freqs.get((word, 0.0),0)
        
#     ### END CODE HERE ###
#     assert(x.shape == (1, 3))
#     return x

In [25]:
# # Check your function
# # test on training data
# tmp1 = extract_features(x_train[1], freqs)
# print(tmp1)

[[1.0000e+00 7.2704e+04 8.5956e+04]]


In [26]:
# # test 2:
# # check for when the words are not in the freqs dictionary
# tmp2 = extract_features('i love it', freqs)
# print(tmp2)

[[1.0000e+00 4.1134e+04 1.4776e+04]]


In [28]:
# len(x_train),len(y_train)

(1119999, 1119999)

In [41]:
# x_train,x_train.tolist()[2]

(1483719                                 @hasselgren Me like 
 796045     The audition comes here in exactly a week but ...
 1433966             L train Canarsie Rockaway Prkway Bound. 
 1084800    @ooohalby 2 appetizers (not on the menu). 2 sa...
 643147     Tweet4Skype could be a nice impementation, but...
                                  ...                        
 1113474                                  @ginavon thank you 
 489373     @FaketragedyCom Yeah. Sigh.  But, off tomorrow...
 1366625    @fujimama your squirrel is kawaii des! Bug is ...
 1213890    @ddlovato andy samberg is gonna sing mother lo...
 188827     I'M GRADUATING! I'M GRADUATING! IT'S ABOUT F'I...
 Name: tweets, Length: 1119999, dtype: object,
 'L train Canarsie Rockaway Prkway Bound. ')

In [43]:
# m = len(x_train)
# x_train = x_train.tolist()
# X = np.zeros((m,3))
# for i in range(m):
# #     p_tweet = process_tweet(tweets[i])
#     X[i,:] = extract_features(x_train[i],freqs)

In [44]:
# print('hello')

hello


In [45]:
# X[:5]

array([[1.0000e+00, 2.7621e+04, 2.9991e+04],
       [1.0000e+00, 6.3883e+04, 8.1751e+04],
       [1.0000e+00, 3.4920e+03, 2.8860e+03],
       [1.0000e+00, 3.0292e+04, 3.8546e+04],
       [1.0000e+00, 4.6316e+04, 6.0816e+04]])

In [46]:
# def sigmoid(z):
#     return 1/(1+np.exp(-z))

In [60]:
# def gradientDescent(x, y, theta, alpha, num_iters):
#     '''
#     Input:
#         x: matrix of features which is (m,n+1)
#         y: corresponding labels of the input matrix x, dimensions (m,1)
#         theta: weight vector of dimension (n+1,1)
#         alpha: learning rate
#         num_iters: number of iterations you want to train your model for
#     Output:
#         J: the final cost
#         theta: your final weight vector
#     Hint: you might want to print the cost to make sure that it is going down.
#     '''
    
#     # get 'm', the number of rows in matrix x
#     m = x.shape[0]
    
#     for i in range(0, num_iters):
        
#         # get z, the dot product of x and theta
#         z = np.dot(x,theta)
        
#         # get the sigmoid of h
#         h = sigmoid(z)
        
#         # calculate the cost function
#         # note that we can use also np.array.transpose() instead of np.array.T
#         # np.array.T just makes code a little more readable :)
#         J = -1./m * (np.dot(y.T, np.log(h)) + np.dot((1-y).T,np.log(1-h)))                                                    

#         # update the weights theta
#         theta = theta - (alpha/m) * np.dot(x.T,(h-y))
        
    
# #     J = float(J)
#     return J, theta

In [68]:
# J, theta = gradientDescent(X, np.array(y_train.tolist()), np.zeros((3, 1)), 1e-3, 2)


MemoryError: Unable to allocate 9.13 TiB for an array with shape (1119999, 1119999) and data type float64

In [67]:
# theta.shape

(3, 500)